In [ ]:
!pip install qrcode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# dependency code

from PIL import Image
import qrcode

def generate_gradient(
        colour1: str, colour2: str, width: int, height: int) -> Image:
    """Generate a vertical gradient."""
    base = Image.new('RGB', (width, height), colour1)
    top = Image.new('RGB', (width, height), colour2)
    mask = Image.new('L', (width, height))
    mask_data = []
    for y in range(height):
        mask_data.extend([int(255 * (y / height))] * width)
    mask.putdata(mask_data)
    base.paste(top, (0, 0), mask)
    return base

def getqr(link):
  img = qrcode.make(link)
  return img
  

In [2]:

# data template csv generator

import requests

r = requests.get("https://raw.githubusercontent.com/ghosh/uiGradients/master/gradients.json")
list_of_grad = r.json()

import pandas as pd

df = pd.DataFrame(columns=["tid","g1","g2","url","name","nft_save_name","teamName","walletAddress"])

gcounter = 0

for i in range(45):
  for j in range(4):
    gradient = list_of_grad[gcounter]["colors"]
    df = df.append({
        "tid":i,"g1":gradient[0],"g2":gradient[1],"url":f"https://verify.hack.kjsce.com/verify/0xabcabc{i}{j}",
        "name":f"firstname{i} {j} lastname{i} {j}","nft_save_name":f"uniquename{i} {j}","teamName":f"teamname{i}",
        "walletAddress": f"0xabcabc{i}{j}"
    },ignore_index=True)
  gcounter+=1

df.to_csv("NFT_team_data_template.csv",index=False)

In [ ]:

# NFT image generator

import pandas as pd

df = pd.read_csv("drive/MyDrive/NFT_team_data.csv") 

guy_lap = Image.open("drive/MyDrive/final_NFT.png").convert("RGBA")



for i in range(len(df)):
  data = df.loc[i]
  base = generate_gradient(data["g1"],data["g2"],1080,1080)
  t = guy_lap.copy()
  base.paste(t,mask=t)
  qr = getqr(data["url"])
  qr = qr.resize((300,300))
  base.paste(qr,(390,671))

  base.save("drive/MyDrive/NFToutput/"+data["nft_save_name"]+".png")



In [ ]:

# metadata generator

import json

urlprefix = "ipfs://img_folder_CID/"


for i in range(len(df)):
  data = df.loc[i]
  metadata = {
    "name": data["name"]+"'s NFT",
    "description": data["name"]+"'s NFT of team "+data["teamName"],
    "image": urlprefix+data["nft_save_name"]+".png",
    "attributes": [
      {
        "trait_type": "team name",
        "value": data["teamName"]
      }
    ]
  }
  with open("drive/MyDrive/NFTmetadata/"+data["nft_save_name"]+".json", "w+") as outfile:
    json.dump(metadata, outfile)





In [ ]:

# json list for minting NFT

df = df[['walletAddress', 'nft_save_name']]

df.to_json("drive/MyDrive/mintlist.json",orient="records")
